# Sprint10
## ディープニューラルネットワークスクラッチ

データ取得と前処理

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
X_train=X_train.reshape(-1,784)
X_test=X_test.reshape(-1,784)

In [5]:
X_train=X_train.astype(np.float)/255
X_test=X_test.astype(np.float)/255

In [6]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2)

In [7]:
enc=OneHotEncoder(handle_unknown='ignore',sparse=False)
y_train_one_hot=enc.fit_transform(y_train[:,np.newaxis])
y_test_one_hot=enc.transform(y_test[:,np.newaxis])

バッチクラス導入

In [8]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練用データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop
    
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]
    

## 【問題1】全結合層のクラス化

In [9]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.s_init = initializer
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.W = self.s_init.W(self.n_nodes1, self.n_nodes2)
        self.B = self.s_init.B(self.n_nodes2)
    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """       
        self.z=X
        A=np.dot(X,self.W)+self.B
        
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dZ=np.dot(dA,self.W.T)
        
        self.dW=np.dot(self.z.T,dA)
        self.dB=dA.sum(axis=0)
        
        # 更新
        self = self.optimizer.update(self)
        return dZ

## 【問題2】初期化方法のクラス化

In [10]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma=0.01):
        self.sigma = sigma
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W=self.sigma*np.random.randn(n_nodes1,n_nodes2)
        
        return W
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B=self.sigma*np.random.randint(n_nodes2)
        return B

## 【問題3】最適化手法のクラス化

In [11]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        
        layer.W-=self.lr*layer.dW
        layer.B-=self.lr*layer.dB
         

## 【問題4】活性化関数のクラス化

In [12]:
class Tanh:
    """
    ハイパボリックタンジェント関数
    """
      
    def forward(self,A):
        self.A=A
        Z=self._Tanh(self.A)
        return Z
    
    def backward(self,Z):
        dA=Z*(1-self._Tanh(self.A)**2)
        return dA
    
    def _Tanh(self,X):
        Z=(np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))
        return Z

In [13]:
class Softmax:
    """
    ソフトマックス関数
    """
    
    def forward(self,A):
        Z=self._softmax(A)
        
        
        return Z
    
    def backward(self,Z,y):
        dA=Z-y
        return dA
    
    def _softmax(self,A):
        Z=np.zeros((A.shape[0],A.shape[1]))
        C=np.max(A)
        exp=np.exp(A/C)
        exp_sum=np.sum(exp,axis=1)
        
        for i in range(A.shape[0]):
            Z[i]=exp[i]/exp_sum[i]
        return Z
        

## 【問題5】ReLUクラスの作成

In [14]:
class ReLU:
    """
    活性化関数(ReLU)クラス
    """
    
    def forward(self,A):
        self.A=A
        return np.maximum(0,A)
        
    def backward(self,Z):
        return Z*np.maximum(np.sin(self.A),0)

## 【問題6】重みの初期値

In [15]:
class XavierInitializer:
    """
    XavierInitializerクラス
    ※シグモイド関数やハイパボリックタンジェント関数のとき
    """
    def __init__(self,n_nodes1):
        self.sigma=1/np.sqrt(n_nodes1)
       
    def W(self, n_nodes1, n_nodes2):
        W=self.sigma*np.random.randn(n_nodes1,n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B=self.sigma*np.random.randint(n_nodes2)
        return B

In [16]:
class HeInitializer:
    """
    HeInitializerクラス
    ※ReLUのとき
    """
    def __init__(self,n_nodes1):
        self.sigma=np.sqrt(2/n_nodes1)
    
    def W(self, n_nodes1, n_nodes2):
        W=self.sigma*np.random.randn(n_nodes1,n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B=self.sigma*np.random.randint(n_nodes2)
        return B

## 【問題7】最適化手法

In [17]:
class AdaGrad:
    """
    最適化手法（AdaGrad）
    ある層の重みやバイアスの更新
    Parameters
    ----------
    layer : 更新前の層のインスタンス
    """
    def __init__(self,lr=0.01):
        self.lr=lr
        self.hW=1
        self.hB=1
        
    def update(self, layer):
        self.hW+=layer.dW
        self.hB+=layer.dB
        layer.W-=self.lr*(1/np.sqrt(self.hW))*layer.dW
        layer.B-=self.lr*(1/np.sqrt(self.hB))*layer.dB
         

## 【問題8】クラスの完成

In [21]:
class ScratchDeepNeuralNetrowkClassifier:
    """
    三層ニューラルネットワーク分類器

    Parameters
    ----------
    

    Attributes
    ----------
    """
    
    def __init__(self,n_nodes1=400,n_nodes2=200,optimizer=SGD,initializer=SimpleInitializer,
                 lr=0.01,sigma=0.01,epochs=50,batch_size=20,verbose=True,seed=9):
        self.lr=lr
        self.sigma=sigma
        self.epochs=epochs
        self.verbose = verbose
        self.n_nodes1 = n_nodes1 # 1層目のノード数
        self.n_nodes2 = n_nodes2 # 2層目のノード数
        self.n_layr=3
        self.optimizer = optimizer(self.lr)
        self.initializer=initializer
        self.batch_size=batch_size
        np.random.seed(seed)
        
        
    def fit(self,X,y):
        """
        学習
        parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練用データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練用データの正解値
        """
        
        self.n_features=X.shape[1]
        self.n_output = y.shape[1] # 出力のクラス数（3層目のノード数）
        self.FC1 = FC(self.n_features, self.n_nodes1, SimpleInitializer(self.sigma), self.optimizer)
        self.activation1 = Tanh()
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, SimpleInitializer(self.sigma), self.optimizer)
        self.activation2 = Tanh()
        self.FC3 = FC(self.n_nodes2, self.n_output, SimpleInitializer(self.sigma), self.optimizer)
        self.activation3 = Softmax()
        self.get_mini_batch=GetMiniBatch(X,y,self.batch_size)
        self.loss=[]
        
        #エポック回数分学習
        for i in range(self.epochs):
            for mini_X_train,mini_y_train in self.get_mini_batch:    
                #forward
                Z=self._forward(mini_X_train)
                #backward
                self._backward(Z)
                 
            #lossの記録
            if self.verbose:
                #verboseをTrueにした際は学習過程などを出力する
                self.loss.append(self._cross(X,y))
                print("{}epoch:{}".format(i+1,self.loss[-1]))
                
    def predict(self,X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        y_pred=np.ardmax(self._forward(X),axis=1)
        
        return y_pred 
        
    def _forward(self,X):
        self.A1 = self.FC1.forward(X)
        self.Z1 = self.activation1.forward(self.A1)
        self.A2 = self.FC2.forward(self.Z1)
        self.Z2 = self.activation2.forward(self.A2)
        self.A3 = self.FC3.forward(self.Z2)
        self.Z3 = self.activation3.forward(self.A3)
        
        return self.Z3
    
    def _backward(self,y):
        dA3 = self.activation3.backward(self.Z3, y) # 交差エントロピー誤差とソフトマックスを合わせている
        dZ2 = self.FC3.backward(dA3)
        dA2 = self.activation2.backward(dZ2)
        dZ1 = self.FC2.backward(dA2)
        dA1 = self.activation1.backward(dZ1)
        dZ0 = self.FC1.backward(dA1) # dZ0は使用しない
        
    def _cross(self,X,y):
        l=-(1/X.shape[0])*np.sum(y*np.log(self._forward(X)))
        
        return l 
    

## 【問題9】学習と推定

In [22]:
DN=ScratchDeepNeuralNetrowkClassifier()

In [23]:
DN.fit(X_train,y_train_one_hot)

1epoch:2.3557647936197363
2epoch:2.3557647936197363
3epoch:2.3557647936197363
4epoch:2.3557647936197363
5epoch:2.3557647936197363
6epoch:2.3557647936197363
7epoch:2.3557647936197363
8epoch:2.3557647936197363
9epoch:2.3557647936197363
10epoch:2.3557647936197363
11epoch:2.3557647936197363
12epoch:2.3557647936197363
13epoch:2.3557647936197363
14epoch:2.3557647936197363
15epoch:2.3557647936197363
16epoch:2.3557647936197363
17epoch:2.3557647936197363
18epoch:2.3557647936197363
19epoch:2.3557647936197363
20epoch:2.3557647936197363
21epoch:2.3557647936197363
22epoch:2.3557647936197363
23epoch:2.3557647936197363
24epoch:2.3557647936197363
25epoch:2.3557647936197363
26epoch:2.3557647936197363
27epoch:2.3557647936197363
28epoch:2.3557647936197363
29epoch:2.3557647936197363
30epoch:2.3557647936197363
31epoch:2.3557647936197363
32epoch:2.3557647936197363
33epoch:2.3557647936197363
34epoch:2.3557647936197363
35epoch:2.3557647936197363
36epoch:2.3557647936197363
37epoch:2.3557647936197363
38epoch:2.